In [2]:
import pandas as pd
import numpy as np
import datetime as DT
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score
import random

# Comenzamos con los datos de genero y edad

Concatenamos los datos disponibles en un solo DF

In [3]:
concatAge = pd.concat([pd.read_csv('data/datos_navent_fiuba/fiuba_2_postulantes_genero_y_edad.csv'),
                       pd.read_csv('data/entrega6/fiuba_2_postulantes_genero_y_edad.csv'),
                       pd.read_csv('data/Fiuba desde 15 Abril/fiuba_2_postulantes_genero_y_edad.csv')]
                    )
concatAge.head()

,idpostulante,fechanacimiento,sexo
0,NM5M,1970-12-03,FEM
1,5awk,1962-12-04,FEM
2,ZaO5,1978-08-10,FEM
3,NdJl,1969-05-09,MASC
4,eo2p,1981-02-16,MASC


In [4]:
concatAge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 780020 entries, 0 to 281384
Data columns (total 3 columns):
idpostulante       780020 non-null object
fechanacimiento    745747 non-null object
sexo               780020 non-null object
dtypes: object(3)
memory usage: 23.8+ MB


Eliminamos los datos duplicados, quedandonos con el ultimo disponible. Ademas calculamos
la edad de cada usuario en base a su fecha de nacimiento. 

In [5]:
concatAge.drop_duplicates('idpostulante',keep = 'last', inplace = True)
concatAge.reset_index(inplace = True)
concatAge.drop(columns = ['index'],inplace = True)
concatAge.rename(columns = {'fechanacimiento':'edad'}, inplace = True)


# hoy : 30/5/2018
hoy = pd.Timestamp(DT.datetime.now())
concatAge['edad'] = pd.to_datetime(concatAge['edad'],errors = 'coerce')
concatAge['edad'] = (hoy - concatAge['edad']).astype('<m8[Y]')

In [6]:
concatAge['sexo'].value_counts()

FEM           251205
MASC          227853
NO_DECLARA     25342
0.0                7
Name: sexo, dtype: int64

Transformamos la columnas del genero de los usuarios al tipo booleano, indicando si son o no
usuarios masculinos. A los que no declararon sexo se les otorga un sexo random (hay que mejorar esto). Por suerte es una cantidad acotada (5%)

In [7]:
def sexoABool(sexo):
    if (sexo == 'MASC'):
        return True
    if (sexo == 'FEM'):
        return False
    return (bool(random.getrandbits(1)))

In [8]:
concatAge['sexo'] = concatAge['sexo'].apply(sexoABool)
concatAge.rename(columns = {'sexo':'sexo_masculino'}, inplace = True)
concatAge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 504407 entries, 0 to 504406
Data columns (total 3 columns):
idpostulante      504407 non-null object
edad              478311 non-null float64
sexo_masculino    504407 non-null bool
dtypes: bool(1), float64(1), object(1)
memory usage: 8.2+ MB


Con la edad sucede algo parecido en cuanto a los datos faltantes. Los rellenamos con numeros
cuya probabilidad corresponden a una distribucion exponencial, manteniendo la media original.

In [9]:
media = concatAge['edad'].mean()
concatAge['edad'].fillna(int(random.expovariate(1/media)),inplace = True)

In [10]:
def recalcularEdad(edad):
    if ((edad < 18) or (edad > 75)):
        nueva_edad = int(random.expovariate(1/media))
        while ((nueva_edad < 18) or (nueva_edad>75)):
            nueva_edad = int(random.expovariate(1/media))
        return nueva_edad
    return edad

In [11]:
concatAge['edad'] = concatAge['edad'].apply(recalcularEdad)

In [12]:
concatAge.edad.value_counts()

32.0    43303
26.0    26396
25.0    25899
27.0    25858
24.0    25491
23.0    24971
22.0    23292
28.0    23126
29.0    21942
21.0    21325
30.0    20026
31.0    19020
20.0    17932
33.0    15150
34.0    14128
19.0    13594
35.0    13149
36.0    12508
37.0    11678
38.0    10806
39.0    10060
40.0     9018
41.0     8043
42.0     7301
18.0     6960
43.0     6592
44.0     5609
45.0     4922
46.0     4563
47.0     4114
48.0     3662
49.0     3116
50.0     2809
51.0     2493
52.0     2226
53.0     2050
54.0     1803
55.0     1627
56.0     1546
57.0     1341
58.0     1100
59.0      887
60.0      669
61.0      611
62.0      428
63.0      310
64.0      233
65.0      171
66.0      146
67.0      130
68.0       86
69.0       56
70.0       37
71.0       24
72.0       23
73.0       19
75.0       14
74.0       14
Name: edad, dtype: int64

# Pasamos a los datos sobre la educacion

Concatenamos todos los datos disponibles sobre la educacion de los usuarios

In [13]:
concatEduc = pd.concat([pd.read_csv('data/datos_navent_fiuba/fiuba_1_postulantes_educacion.csv'),
                        pd.read_csv('data/entrega6/fiuba_1_postulantes_educacion.csv'),
                        pd.read_csv('data/Fiuba desde 15 Abril/fiuba_1_postulantes_educacion.csv')]
                      )
concatEduc.head()

,idpostulante,nombre,estado
0,NdJl,Posgrado,En Curso
1,8BkL,Universitario,En Curso
2,1d2B,Universitario,En Curso
3,NPBx,Universitario,En Curso
4,NPBx,Master,En Curso


In [14]:
concatEduc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1103193 entries, 0 to 397508
Data columns (total 3 columns):
idpostulante    1103193 non-null object
nombre          1103193 non-null object
estado          1103193 non-null object
dtypes: object(3)
memory usage: 33.7+ MB


In [15]:
concatEduc['nombre'].value_counts()

Universitario        404619
Secundario           382927
Terciario/Técnico    180091
Otro                  84674
Posgrado              33484
Master                16436
Doctorado               962
Name: nombre, dtype: int64

Agregamos columnas que indiquen si el usuario está graduado en educacion de determinado
nivel

In [16]:
def estudioSuperior(nivel):
    if ((nivel=='Posgrado') or (nivel =='Doctorado') or (nivel=='Master')):
        return 1
    else:
        return 0

In [17]:
graduados = (concatEduc['estado'].astype(str)== 'Graduado').astype(int)

universitarios = (concatEduc['nombre'].astype(str) == 'Universitario').astype(int)

secundario = (concatEduc['nombre'].astype(str) == 'Secundario').astype(int)

mayor_a_universitario = concatEduc['nombre'].apply(estudioSuperior)

In [18]:
concatEduc['titulo_universitario'] = universitarios*graduados
concatEduc['titulo_secundario'] = secundario*graduados
concatEduc['titulo_superior'] = mayor_a_universitario * graduados

In [19]:
titulos = concatEduc.groupby('idpostulante').agg({'titulo_universitario':'max',
                                                   'titulo_secundario':'max',
                                                   'titulo_superior':'max'})
titulos.reset_index(inplace = True)

titulos.head()

,idpostulante,titulo_universitario,titulo_secundario,titulo_superior
0,0z5Dmrd,1,0,0
1,0z5JW1r,0,1,0
2,0z5VvGv,0,1,0
3,0zB01pE,0,0,0
4,0zB026d,0,1,0


Aqui creamos un nuevo DF llamado 'usuarios', donde cada fila corresponde a un usuario y 
posee las columnas que corresponden a los datos sobre su educacion, edad y genero

In [20]:
usuarios = concatAge.merge(titulos, on = 'idpostulante', how = 'left')
usuarios.head()

,idpostulante,edad,sexo_masculino,titulo_universitario,titulo_secundario,titulo_superior
0,eo2p,37.0,True,1.0,1.0,1.0
1,1d2B,42.0,True,0.0,0.0,0.0
2,EBO0,44.0,False,1.0,0.0,1.0
3,a6MKW,44.0,True,1.0,0.0,1.0
4,6MWd4,43.0,True,1.0,0.0,0.0


convertimos a booleano algunas columnas para reducir tamaño

In [21]:
usuarios['titulo_universitario'] = usuarios['titulo_universitario'].astype(bool)
usuarios['titulo_secundario'] = usuarios['titulo_secundario'].astype(bool)
usuarios['titulo_superior'] = usuarios['titulo_superior'].astype(bool)

In [22]:
usuarios.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 504407 entries, 0 to 504406
Data columns (total 6 columns):
idpostulante            504407 non-null object
edad                    504407 non-null float64
sexo_masculino          504407 non-null bool
titulo_universitario    504407 non-null bool
titulo_secundario       504407 non-null bool
titulo_superior         504407 non-null bool
dtypes: bool(4), float64(1), object(1)
memory usage: 13.5+ MB


# Pasamos a los datos sobre los avisos

Concatenamos toda la informacion sobre avisos

In [23]:
avisos = pd.concat([pd.read_csv('data/datos_navent_fiuba/fiuba_6_avisos_detalle.csv'),
                    pd.read_csv('data/entrega6/fiuba_6_avisos_detalle.csv'),
                    pd.read_csv('data/Fiuba desde 15 Abril/fiuba_6_avisos_detalle.csv'),
                    pd.read_csv('data/fiuba_6_avisos_detalle_missing_nivel_laboral.csv')])


avisos.head()

,idaviso,idpais,titulo,descripcion,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,8725750,1,VENDEDOR/A PROVINCIA DE SANTA FE,<p><strong><strong>Empresa:</strong></strong> ...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Comercial,VENTOR
1,17903700,1,Enfermeras,<p>Solicitamos para importante cadena de farma...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Salud,Farmacias Central Oeste
2,1000150677,1,Chofer de taxi,<p>TE GUSTA MANEJAR? QUERES GANAR PLATA HACIEN...,Capital Federal,NaN,Empedrado 2336,Full-time,Senior / Semi-Senior,Transporte,FAMITAX SRL
3,1000610287,1,CHOFER DE CAMIONETA BAHIA BLANCA - PUNTA ALTA,<p><strong>Somos una empresa multinacional que...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Transporte,Wurth Argentina S.A
4,1000872556,1,Operarios de Planta - Rubro Electrodomésticos,<p><strong>OPERARIOS DE PLANTA</strong></p><p>...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Producción,ELECTRO OUTLET SRL


In [25]:
avisos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45969 entries, 0 to 337
Data columns (total 11 columns):
idaviso                 45969 non-null int64
idpais                  45969 non-null int64
titulo                  45969 non-null object
descripcion             45969 non-null object
nombre_zona             45969 non-null object
ciudad                  225 non-null object
mapacalle               3444 non-null object
tipo_de_trabajo         45969 non-null object
nivel_laboral           45634 non-null object
nombre_area             45969 non-null object
denominacion_empresa    45955 non-null object
dtypes: int64(2), object(9)
memory usage: 4.2+ MB


Eliminamos los datos repetidos, quedándonos con el último disponible

In [26]:
avisos.drop_duplicates('idaviso', keep = 'last', inplace = True)
avisos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25288 entries, 4 to 337
Data columns (total 11 columns):
idaviso                 25288 non-null int64
idpais                  25288 non-null int64
titulo                  25288 non-null object
descripcion             25288 non-null object
nombre_zona             25288 non-null object
ciudad                  160 non-null object
mapacalle               2070 non-null object
tipo_de_trabajo         25288 non-null object
nivel_laboral           24953 non-null object
nombre_area             25288 non-null object
denominacion_empresa    25281 non-null object
dtypes: int64(2), object(9)
memory usage: 2.3+ MB


In [27]:
avisos.idpais.value_counts()

1    25288
Name: idpais, dtype: int64

Eliminamos las columnas cuyos elementos son casi todos nulos, o cuyos datos son todos iguales

In [28]:
avisos.drop(columns = ['mapacalle','ciudad','idpais'], inplace = True)

In [29]:
avisos['tipo_de_trabajo'].value_counts()

Full-time          22831
Part-time           1746
Teletrabajo          248
Por Horas            125
Pasantia             119
Temporario            96
Por Contrato          88
Fines de Semana       28
Primer empleo          6
Voluntario             1
Name: tipo_de_trabajo, dtype: int64

Vemos que la mayoria de los tipos de trabajo son full o part-time. Creamos dos columnas que nos indiquen si el trabajo es Full-time y si es Part-time

In [30]:
avisos['Full-time'] = (avisos['tipo_de_trabajo'].astype(str) == 'Full-time').astype(bool)
avisos['Part-time'] = (avisos['tipo_de_trabajo'].astype(str) == 'Part-time').astype(bool)

In [31]:
avisos['nombre_zona'].value_counts()

Gran Buenos Aires              23017
Capital Federal                 2183
Buenos Aires (fuera de GBA)       57
GBA Oeste                         10
La Plata                           4
Mendoza                            3
Ciudad de Mendoza                  3
Cordoba                            2
Rosario                            2
Tucuman                            1
Corrientes                         1
Santa Cruz                         1
Santa Fe                           1
Neuquen                            1
San Juan                           1
Catamarca                          1
Name: nombre_zona, dtype: int64

Casi el 100% de los avisos son de GBA o capital federal. Creamos una columna booleanaque indique si el trabajo es en GBA

In [32]:
avisos.rename(columns = {'nombre_zona':'GBA'},inplace = True)

In [33]:
avisos['GBA'] = (avisos['GBA'].astype(str) == 'Gran Buenos Aires').astype(bool)
avisos.head(10)

,idaviso,titulo,descripcion,GBA,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,Full-time,Part-time
4,1000872556,Operarios de Planta - Rubro Electrodomésticos,<p><strong>OPERARIOS DE PLANTA</strong></p><p>...,True,Full-time,Senior / Semi-Senior,Producción,ELECTRO OUTLET SRL,True,False
8,9240880,"Productores Asesores Independiente, para venta...",Agente\r\noficial Selecciona:</span></strong><...,False,Full-time,Jefe / Supervisor / Responsable,Comercial,Agencia Oficial Alejandro Arizaga,True,False
19,1110185164,Administrativo de Recepción,<p>Centro Médico Accord se encuentra en la bús...,True,Full-time,Junior,Administración,Unión Personal,True,False
21,1110513885,Distribuidor domiciliario con moto (SAN MIGUEL),<p>Importante correo privado ubicado en SAN MI...,True,Full-time,Junior,Distribución,Asoko Tempo SA,True,False
32,1111034024,Vendedores para venta de medicina pre paga y o...,<p> </p><p><strong>Comercializadora incorpora ...,True,Full-time,Senior / Semi-Senior,Comercial,JELS SRL,True,False
35,1111101289,PM. Zona Norte / Pacheco,<p>Acciona IT se encuentra en la búsqueda de P...,True,Full-time,Senior / Semi-Senior,Liderazgo de Proyecto,ACCIONA IT,True,False
38,1111109704,"Supervisor de obra civil, eléctrico/ instrumen...",<p>Buscamos para importante empresa petrolera ...,True,Full-time,Senior / Semi-Senior,Construcción,Hahn Solarz S.R.L.,True,False
45,1111172357,Empleado Administrativo para Tramites de Habil...,"<p style=""""><strong><em><span style="""">Brujula...",True,Full-time,Senior / Semi-Senior,Administración,Brujula SA,True,False
46,1111174081,Gestor de Cobranzas - Telecobrador,<p><strong>En MAS ACTIVOS BPO te estamos esper...,False,Part-time,Senior / Semi-Senior,Call Center,MAS ACTIVOS S.A.,False,True
47,1111235995,Analista de Recursos Humanos - Quilmes (Plazo...,<p>Cervecería y Maltería Quilmes es una de las...,True,Full-time,Senior / Semi-Senior,Recursos Humanos,AB InBev - Cervecería y Maltería Quilmes,True,False


La columna de tipo de trabajo ya no es relevante, la dropeamos.

In [34]:
avisos.drop(columns = 'tipo_de_trabajo', inplace = True)

In [35]:
avisos['nivel_laboral'].value_counts()

Senior / Semi-Senior                    16975
Junior                                   4152
Otro                                     1977
Jefe / Supervisor / Responsable          1527
Gerencia / Alta Gerencia / Dirección      322
Name: nivel_laboral, dtype: int64

Transformamos la columna de niuvel laborar en 5 features que indiquen si el puesto de trabajo
corresponde o no a determinado nivel

In [36]:
avisos.rename(columns = {'nivel_laboral':'nivel'}, inplace = True)


In [37]:
avisos['senior'] = (avisos['nivel'].astype(str) == 'Senior / Semi-Senior').astype(bool)
avisos['junior'] = (avisos['nivel'].astype(str) == 'Junior').astype(bool)
avisos['otro'] = (avisos['nivel'].astype(str) == 'Otro').astype(bool)
avisos['Jefe'] = (avisos['nivel'].astype(str) == 'Jefe / Supervisor / Responsable').astype(bool)
avisos['Gerencia'] = (avisos['nivel'].astype(str) == 'Gerencia')

In [38]:
avisos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25288 entries, 4 to 337
Data columns (total 14 columns):
idaviso                 25288 non-null int64
titulo                  25288 non-null object
descripcion             25288 non-null object
GBA                     25288 non-null bool
nivel                   24953 non-null object
nombre_area             25288 non-null object
denominacion_empresa    25281 non-null object
Full-time               25288 non-null bool
Part-time               25288 non-null bool
senior                  25288 non-null bool
junior                  25288 non-null bool
otro                    25288 non-null bool
Jefe                    25288 non-null bool
Gerencia                25288 non-null bool
dtypes: bool(8), int64(1), object(5)
memory usage: 1.5+ MB


In [39]:
avisos['nombre_area'].value_counts()

Ventas                                       3153
Comercial                                    1780
Administración                               1708
Producción                                   1536
Programación                                  958
Contabilidad                                  783
Tecnologia / Sistemas                         758
Atención al Cliente                           657
Mantenimiento                                 537
Recursos Humanos                              423
Logística                                     400
Gastronomia                                   394
Call Center                                   391
Soporte Técnico                               382
Almacén / Depósito / Expedición               379
Oficios y Profesiones                         377
Marketing                                     321
Recepcionista                                 306
Mantenimiento y Limpieza                      295
Compras                                       287


Agregamos features que indiquen si el aviso corresponde a una determinada area laboral, para
las N areas laborales que querramos

In [40]:
top_areas = avisos['nombre_area'].value_counts().nlargest(17).index
for area in top_areas:
    avisos[area] = (avisos['nombre_area'] == area).astype(bool)
    
avisos.head()

,idaviso,titulo,descripcion,GBA,nivel,nombre_area,denominacion_empresa,Full-time,Part-time,senior,...,Atención al Cliente,Mantenimiento,Recursos Humanos,Logística,Gastronomia,Call Center,Soporte Técnico,Almacén / Depósito / Expedición,Oficios y Profesiones,Marketing
4,1000872556,Operarios de Planta - Rubro Electrodomésticos,<p><strong>OPERARIOS DE PLANTA</strong></p><p>...,True,Senior / Semi-Senior,Producción,ELECTRO OUTLET SRL,True,False,True,...,False,False,False,False,False,False,False,False,False,False
8,9240880,"Productores Asesores Independiente, para venta...",Agente\r\noficial Selecciona:</span></strong><...,False,Jefe / Supervisor / Responsable,Comercial,Agencia Oficial Alejandro Arizaga,True,False,False,...,False,False,False,False,False,False,False,False,False,False
19,1110185164,Administrativo de Recepción,<p>Centro Médico Accord se encuentra en la bús...,True,Junior,Administración,Unión Personal,True,False,False,...,False,False,False,False,False,False,False,False,False,False
21,1110513885,Distribuidor domiciliario con moto (SAN MIGUEL),<p>Importante correo privado ubicado en SAN MI...,True,Junior,Distribución,Asoko Tempo SA,True,False,False,...,False,False,False,False,False,False,False,False,False,False
32,1111034024,Vendedores para venta de medicina pre paga y o...,<p> </p><p><strong>Comercializadora incorpora ...,True,Senior / Semi-Senior,Comercial,JELS SRL,True,False,True,...,False,False,False,False,False,False,False,False,False,False


Dropeamos algunas columnas que ya no nos interesan mas

In [41]:
avisos.drop(columns = ['titulo','descripcion','denominacion_empresa','nivel'],
            inplace = True)

In [42]:
avisos.head()

,idaviso,GBA,nombre_area,Full-time,Part-time,senior,junior,otro,Jefe,Gerencia,...,Atención al Cliente,Mantenimiento,Recursos Humanos,Logística,Gastronomia,Call Center,Soporte Técnico,Almacén / Depósito / Expedición,Oficios y Profesiones,Marketing
4,1000872556,True,Producción,True,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8,9240880,False,Comercial,True,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
19,1110185164,True,Administración,True,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
21,1110513885,True,Distribución,True,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
32,1111034024,True,Comercial,True,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [43]:
avisos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25288 entries, 4 to 337
Data columns (total 27 columns):
idaviso                            25288 non-null int64
GBA                                25288 non-null bool
nombre_area                        25288 non-null object
Full-time                          25288 non-null bool
Part-time                          25288 non-null bool
senior                             25288 non-null bool
junior                             25288 non-null bool
otro                               25288 non-null bool
Jefe                               25288 non-null bool
Gerencia                           25288 non-null bool
Ventas                             25288 non-null bool
Comercial                          25288 non-null bool
Administración                     25288 non-null bool
Producción                         25288 non-null bool
Programación                       25288 non-null bool
Contabilidad                       25288 non-null bool
Tecnologia

# Vistas

Concatenamos todos los datos de vistas en un DF

In [44]:
concatVistas = pd.concat([pd.read_csv('data/datos_navent_fiuba/fiuba_3_vistas.csv'),
                          pd.read_csv('data/entrega6/fiuba_3_vistas.csv'),
                          pd.read_csv('data/Fiuba desde 15 Abril/fiuba_3_vistas.csv')])

concatVistas.head()

,idAviso,timestamp,idpostulante
0,1111780242,2018-02-23T13:38:13.187-0500,YjVJQ6Z
1,1112263876,2018-02-23T13:38:14.296-0500,BmVpYoR
2,1112327963,2018-02-23T13:38:14.329-0500,wVkBzZd
3,1112318643,2018-02-23T13:38:17.921-0500,OqmP9pv
4,1111903673,2018-02-23T13:38:18.973-0500,DrpbXDP


In [45]:
concatVistas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18368813 entries, 0 to 11648229
Data columns (total 3 columns):
idAviso         int64
timestamp       object
idpostulante    object
dtypes: int64(1), object(2)
memory usage: 560.6+ MB


En vez guardar un registro de cada visita realizada, guardamos la cantidad de visitas 
que realizo cada postulante a cada aviso que visitó

In [46]:
concatVistas['visitas'] = 1
concatVistas.rename(columns = {'idAviso':'idaviso'}, inplace = True)

In [47]:
concatVistas = concatVistas.groupby(['idpostulante','idaviso'])\
                           .agg({'visitas':'sum'})\
                           .reset_index()

In [48]:
concatVistas.head()

,idpostulante,idaviso,visitas
0,0002q,1789742,4
1,0002q,1807692,6
2,0002q,1808090,5
3,0005E,1744005,3
4,0005E,1785430,2


In [49]:
concatVistas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8597915 entries, 0 to 8597914
Data columns (total 3 columns):
idpostulante    object
idaviso         int64
visitas         int64
dtypes: int64(2), object(1)
memory usage: 196.8+ MB


# Pasamos a las postulaciones

Concatenamos los datos de postulaciones disponibles

In [50]:
postulaciones = pd.concat([pd.read_csv('data/datos_navent_fiuba/fiuba_4_postulaciones.csv'),
                           pd.read_csv('data/entrega6/fiuba_4_postulaciones.csv')])
postulaciones.head()

,idaviso,idpostulante,fechapostulacion
0,1112257047,NM5M,2018-01-15 16:22:34
1,1111920714,NM5M,2018-02-06 09:04:50
2,1112346945,NM5M,2018-02-22 09:04:47
3,1112345547,NM5M,2018-02-22 09:04:59
4,1112237522,5awk,2018-01-25 18:55:03


Dropeamos la columna de fecha/horario ya que no nos interesa

In [51]:
postulaciones.drop(columns = 'fechapostulacion', inplace = True)

In [52]:
postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8311264 entries, 0 to 4909640
Data columns (total 2 columns):
idaviso         int64
idpostulante    object
dtypes: int64(1), object(1)
memory usage: 190.2+ MB


Eliminamos las postulaciones duplicadas

In [53]:
postulaciones.drop_duplicates(subset = ['idaviso','idpostulante'],
                              keep = 'first',
                              inplace = True)
postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6603752 entries, 0 to 4909640
Data columns (total 2 columns):
idaviso         int64
idpostulante    object
dtypes: int64(1), object(1)
memory usage: 151.1+ MB


### Aca agrego features a los usuarios de acuerdo a las areas a las que postularon

In [54]:
postulaciones = postulaciones.merge(avisos[['idaviso','nombre_area','GBA']],
                                    on  = 'idaviso',
                                    how = 'left')

In [55]:
postulaciones.head()

,idaviso,idpostulante,nombre_area,GBA
0,1112257047,NM5M,Atención al Cliente,True
1,1111920714,NM5M,Telemarketing,True
2,1112346945,NM5M,Telemarketing,True
3,1112345547,NM5M,Telemarketing,True
4,1112237522,5awk,Contabilidad,True


Listamos(manteniendo la multiplicidad), las áreas a las que se postulo cada usuarios

In [56]:
areas_por_postulante = postulaciones.groupby('idpostulante')\
                .agg({'nombre_area': lambda x: list(x),
                      'idaviso':'count',
                      'GBA':'sum'})

areas_por_postulante.head()

,nombre_area,idaviso,GBA
idpostulante,,,
0z5Dmrd,"[Recursos Humanos, Recursos Humanos]",2,2
0z5JW1r,"[Almacén / Depósito / Expedición, Comercial, D...",7,7
0z5VvGv,"[Ventas, Ventas, Atención al Cliente, Atención...",50,44
0zB01pE,"[Administracion de Seguros, Administracion de ...",3,1
0zB026d,"[Producción, Producción, Producción, Producció...",10,10


In [57]:
areas_por_postulante.reset_index(inplace = True)
areas_por_postulante.rename(columns = {'nombre_area':'areas', 'idaviso':'cantidad_postulaciones',
                                      'GBA':'postulaciones_gba'},
                            inplace = True)

areas_por_postulante.head()

,idpostulante,areas,cantidad_postulaciones,postulaciones_gba
0,0z5Dmrd,"[Recursos Humanos, Recursos Humanos]",2,2
1,0z5JW1r,"[Almacén / Depósito / Expedición, Comercial, D...",7,7
2,0z5VvGv,"[Ventas, Ventas, Atención al Cliente, Atención...",50,44
3,0zB01pE,"[Administracion de Seguros, Administracion de ...",3,1
4,0zB026d,"[Producción, Producción, Producción, Producció...",10,10


In [58]:
areas_por_postulante.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 348883 entries, 0 to 348882
Data columns (total 4 columns):
idpostulante              348883 non-null object
areas                     348883 non-null object
cantidad_postulaciones    348883 non-null int64
postulaciones_gba         348883 non-null object
dtypes: int64(1), object(3)
memory usage: 10.6+ MB


Aqui vamos a agregar un feature que indique, por cada una de las top_areas que habiamos
definido cuando trabajamos con los avisos, que indique la cantidad de postulaciones que cada
usuarios realizo a esa area

In [59]:
for area in top_areas:
    lista_area = []
    for i in range(0,len(areas_por_postulante)):
        lista_areas_usuario = areas_por_postulante.loc[i,'areas']
        lista_area.append(lista_areas_usuario.count(area))
    
    areas_por_postulante['postulaciones_'+ area] = pd.Series(lista_area)

In [60]:
areas_por_postulante.head()

,idpostulante,areas,cantidad_postulaciones,postulaciones_gba,postulaciones_Ventas,postulaciones_Comercial,postulaciones_Administración,postulaciones_Producción,postulaciones_Programación,postulaciones_Contabilidad,...,postulaciones_Atención al Cliente,postulaciones_Mantenimiento,postulaciones_Recursos Humanos,postulaciones_Logística,postulaciones_Gastronomia,postulaciones_Call Center,postulaciones_Soporte Técnico,postulaciones_Almacén / Depósito / Expedición,postulaciones_Oficios y Profesiones,postulaciones_Marketing
0,0z5Dmrd,"[Recursos Humanos, Recursos Humanos]",2,2,0,0,0,0,0,0,...,0,0,2,0,0,0,0,0,0,0
1,0z5JW1r,"[Almacén / Depósito / Expedición, Comercial, D...",7,7,1,1,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
2,0z5VvGv,"[Ventas, Ventas, Atención al Cliente, Atención...",50,44,13,7,1,0,0,0,...,11,0,0,1,0,4,0,0,0,0
3,0zB01pE,"[Administracion de Seguros, Administracion de ...",3,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0zB026d,"[Producción, Producción, Producción, Producció...",10,10,1,0,0,8,0,0,...,0,0,0,0,0,0,0,1,0,0


Agregamos al DF de usuarios estas columnas (features) que acabamos de calcular. 

In [61]:
usuarios = usuarios.merge(areas_por_postulante.drop(columns = 'areas'),
                          on = 'idpostulante',
                          how = 'left')
usuarios.head()

,idpostulante,edad,sexo_masculino,titulo_universitario,titulo_secundario,titulo_superior,cantidad_postulaciones,postulaciones_gba,postulaciones_Ventas,postulaciones_Comercial,...,postulaciones_Atención al Cliente,postulaciones_Mantenimiento,postulaciones_Recursos Humanos,postulaciones_Logística,postulaciones_Gastronomia,postulaciones_Call Center,postulaciones_Soporte Técnico,postulaciones_Almacén / Depósito / Expedición,postulaciones_Oficios y Profesiones,postulaciones_Marketing
0,eo2p,37.0,True,True,True,True,6.0,2,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1d2B,42.0,True,False,False,False,4.0,3,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,EBO0,44.0,False,True,False,True,8.0,7,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,a6MKW,44.0,True,True,False,True,3.0,3,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,6MWd4,43.0,True,True,False,False,3.0,3,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [62]:
usuarios.isnull().sum()

idpostulante                                          0
edad                                                  0
sexo_masculino                                        0
titulo_universitario                                  0
titulo_secundario                                     0
titulo_superior                                       0
cantidad_postulaciones                           155524
postulaciones_gba                                155524
postulaciones_Ventas                             155524
postulaciones_Comercial                          155524
postulaciones_Administración                     155524
postulaciones_Producción                         155524
postulaciones_Programación                       155524
postulaciones_Contabilidad                       155524
postulaciones_Tecnologia / Sistemas              155524
postulaciones_Atención al Cliente                155524
postulaciones_Mantenimiento                      155524
postulaciones_Recursos Humanos                  

Tenemos usuarios que no realizaron postulaciones, por eso poseen algunos features nulos, 
los completamos con ceros, que es lo que corresponde en este caso

In [63]:
usuarios.fillna(0, inplace = True)
usuarios.isnull().sum()

idpostulante                                     0
edad                                             0
sexo_masculino                                   0
titulo_universitario                             0
titulo_secundario                                0
titulo_superior                                  0
cantidad_postulaciones                           0
postulaciones_gba                                0
postulaciones_Ventas                             0
postulaciones_Comercial                          0
postulaciones_Administración                     0
postulaciones_Producción                         0
postulaciones_Programación                       0
postulaciones_Contabilidad                       0
postulaciones_Tecnologia / Sistemas              0
postulaciones_Atención al Cliente                0
postulaciones_Mantenimiento                      0
postulaciones_Recursos Humanos                   0
postulaciones_Logística                          0
postulaciones_Gastronomia      

### Postulaciones por aviso

In [64]:
postulaciones.head()

,idaviso,idpostulante,nombre_area,GBA
0,1112257047,NM5M,Atención al Cliente,True
1,1111920714,NM5M,Telemarketing,True
2,1112346945,NM5M,Telemarketing,True
3,1112345547,NM5M,Telemarketing,True
4,1112237522,5awk,Contabilidad,True


In [65]:
postulaciones = postulaciones.merge(usuarios[['idpostulante','sexo_masculino','titulo_universitario']],
                                    on = 'idpostulante', how = 'left')
postulaciones.head()

,idaviso,idpostulante,nombre_area,GBA,sexo_masculino,titulo_universitario
0,1112257047,NM5M,Atención al Cliente,True,False,False
1,1111920714,NM5M,Telemarketing,True,False,False
2,1112346945,NM5M,Telemarketing,True,False,False
3,1112345547,NM5M,Telemarketing,True,False,False
4,1112237522,5awk,Contabilidad,True,False,True


In [66]:
postulaciones_por_aviso = postulaciones.groupby('idaviso')\
                                        .agg({'idpostulante':'count','sexo_masculino':'mean',
                                              'titulo_universitario':'mean'})

In [67]:
postulaciones_por_aviso.reset_index(inplace = True)

In [68]:
postulaciones_por_aviso.head()

,idaviso,idpostulante,sexo_masculino,titulo_universitario
0,8725750,137,0.583942,0.262774
1,11740890,13,0.461538,0.538462
2,12543760,1080,0.426852,0.476852
3,12812680,159,0.119497,0.132075
4,17903700,175,0.108571,0.377143


In [69]:
postulaciones_por_aviso.rename(columns = {'idpostulante':'postulaciones_aviso',
                                          'sexo_masculino':'postulaciones_masculinas',
                                          'titulo_universitario':'postulaciones_universitarias'},
                               inplace = True)
postulaciones_por_aviso.head()

,idaviso,postulaciones_aviso,postulaciones_masculinas,postulaciones_universitarias
0,8725750,137,0.583942,0.262774
1,11740890,13,0.461538,0.538462
2,12543760,1080,0.426852,0.476852
3,12812680,159,0.119497,0.132075
4,17903700,175,0.108571,0.377143


In [70]:
avisos = avisos.merge(postulaciones_por_aviso, on = 'idaviso', how = 'left')
avisos.head()

,idaviso,GBA,nombre_area,Full-time,Part-time,senior,junior,otro,Jefe,Gerencia,...,Logística,Gastronomia,Call Center,Soporte Técnico,Almacén / Depósito / Expedición,Oficios y Profesiones,Marketing,postulaciones_aviso,postulaciones_masculinas,postulaciones_universitarias
0,1000872556,True,Producción,True,False,True,False,False,False,False,...,False,False,False,False,False,False,False,NaN,NaN,NaN
1,9240880,False,Comercial,True,False,False,False,False,True,False,...,False,False,False,False,False,False,False,NaN,NaN,NaN
2,1110185164,True,Administración,True,False,False,True,False,False,False,...,False,False,False,False,False,False,False,NaN,NaN,NaN
3,1110513885,True,Distribución,True,False,False,True,False,False,False,...,False,False,False,False,False,False,False,NaN,NaN,NaN
4,1111034024,True,Comercial,True,False,True,False,False,False,False,...,False,False,False,False,False,False,False,NaN,NaN,NaN


In [71]:
avisos['postulaciones_aviso'].isnull().sum()

6593

In [72]:
avisos['postulaciones_aviso'].fillna(0,inplace = True)
avisos.head()

,idaviso,GBA,nombre_area,Full-time,Part-time,senior,junior,otro,Jefe,Gerencia,...,Logística,Gastronomia,Call Center,Soporte Técnico,Almacén / Depósito / Expedición,Oficios y Profesiones,Marketing,postulaciones_aviso,postulaciones_masculinas,postulaciones_universitarias
0,1000872556,True,Producción,True,False,True,False,False,False,False,...,False,False,False,False,False,False,False,0.0,NaN,NaN
1,9240880,False,Comercial,True,False,False,False,False,True,False,...,False,False,False,False,False,False,False,0.0,NaN,NaN
2,1110185164,True,Administración,True,False,False,True,False,False,False,...,False,False,False,False,False,False,False,0.0,NaN,NaN
3,1110513885,True,Distribución,True,False,False,True,False,False,False,...,False,False,False,False,False,False,False,0.0,NaN,NaN
4,1111034024,True,Comercial,True,False,True,False,False,False,False,...,False,False,False,False,False,False,False,0.0,NaN,NaN


In [73]:
avisos['postulaciones_masculinas'].isnull().sum()

6593

In [74]:
avisos['postulaciones_masculinas'].fillna(0,inplace = True)
avisos['postulaciones_universitarias'].fillna(0,inplace = True)
avisos.head()

,idaviso,GBA,nombre_area,Full-time,Part-time,senior,junior,otro,Jefe,Gerencia,...,Logística,Gastronomia,Call Center,Soporte Técnico,Almacén / Depósito / Expedición,Oficios y Profesiones,Marketing,postulaciones_aviso,postulaciones_masculinas,postulaciones_universitarias
0,1000872556,True,Producción,True,False,True,False,False,False,False,...,False,False,False,False,False,False,False,0.0,0.0,0.0
1,9240880,False,Comercial,True,False,False,False,False,True,False,...,False,False,False,False,False,False,False,0.0,0.0,0.0
2,1110185164,True,Administración,True,False,False,True,False,False,False,...,False,False,False,False,False,False,False,0.0,0.0,0.0
3,1110513885,True,Distribución,True,False,False,True,False,False,False,...,False,False,False,False,False,False,False,0.0,0.0,0.0
4,1111034024,True,Comercial,True,False,True,False,False,False,False,...,False,False,False,False,False,False,False,0.0,0.0,0.0


### Volvemos a las postulaciones

In [75]:
postulaciones.head()

,idaviso,idpostulante,nombre_area,GBA,sexo_masculino,titulo_universitario
0,1112257047,NM5M,Atención al Cliente,True,False,False
1,1111920714,NM5M,Telemarketing,True,False,False
2,1112346945,NM5M,Telemarketing,True,False,False
3,1112345547,NM5M,Telemarketing,True,False,False
4,1112237522,5awk,Contabilidad,True,False,True


A cada postulacion le agregamos los features del aviso en cuestión

In [76]:
postulaciones.drop(columns = ['nombre_area','sexo_masculino','titulo_universitario','GBA'], inplace = True)
postulaciones = postulaciones.merge(avisos, 
                                    on = 'idaviso',
                                    how = 'left')

postulaciones.head()

,idaviso,idpostulante,GBA,nombre_area,Full-time,Part-time,senior,junior,otro,Jefe,...,Logística,Gastronomia,Call Center,Soporte Técnico,Almacén / Depósito / Expedición,Oficios y Profesiones,Marketing,postulaciones_aviso,postulaciones_masculinas,postulaciones_universitarias
0,1112257047,NM5M,True,Atención al Cliente,False,True,False,False,True,False,...,False,False,False,False,False,False,False,2391.0,0.268507,0.080719
1,1111920714,NM5M,True,Telemarketing,False,True,True,False,False,False,...,False,False,False,False,False,False,False,524.0,0.293893,0.076336
2,1112346945,NM5M,True,Telemarketing,True,False,True,False,False,False,...,False,False,False,False,False,False,False,1126.0,0.434281,0.156306
3,1112345547,NM5M,True,Telemarketing,True,False,True,False,False,False,...,False,False,False,False,False,False,False,1804.0,0.334812,0.093126
4,1112237522,5awk,True,Contabilidad,True,False,True,False,False,False,...,False,False,False,False,False,False,False,193.0,0.554404,0.932642


In [77]:
postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6603752 entries, 0 to 6603751
Data columns (total 31 columns):
idaviso                            int64
idpostulante                       object
GBA                                object
nombre_area                        object
Full-time                          object
Part-time                          object
senior                             object
junior                             object
otro                               object
Jefe                               object
Gerencia                           object
Ventas                             object
Comercial                          object
Administración                     object
Producción                         object
Programación                       object
Contabilidad                       object
Tecnologia / Sistemas              object
Atención al Cliente                object
Mantenimiento                      object
Recursos Humanos                   object
Logíst

Agregamos una columna que indique que se realizo una postulación que implica el par
(idpostulante, idaviso). Esto ahora parece no tener sentido, pero luego querremos tener
también pares que no impliquen una postulacion, y de esta forma podremos diferenciarlos

In [78]:
postulaciones['postulacion'] = True
postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6603752 entries, 0 to 6603751
Data columns (total 32 columns):
idaviso                            int64
idpostulante                       object
GBA                                object
nombre_area                        object
Full-time                          object
Part-time                          object
senior                             object
junior                             object
otro                               object
Jefe                               object
Gerencia                           object
Ventas                             object
Comercial                          object
Administración                     object
Producción                         object
Programación                       object
Contabilidad                       object
Tecnologia / Sistemas              object
Atención al Cliente                object
Mantenimiento                      object
Recursos Humanos                   object
Logíst

Ahora agregamos a la postulacion, al igual que agregamos los datos de los avisos, los datos
de los usuarios

In [79]:
postulaciones = postulaciones.merge(usuarios, on = 'idpostulante', how = 'left')
postulaciones.head()

,idaviso,idpostulante,GBA,nombre_area,Full-time,Part-time,senior,junior,otro,Jefe,...,postulaciones_Atención al Cliente,postulaciones_Mantenimiento,postulaciones_Recursos Humanos,postulaciones_Logística,postulaciones_Gastronomia,postulaciones_Call Center,postulaciones_Soporte Técnico,postulaciones_Almacén / Depósito / Expedición,postulaciones_Oficios y Profesiones,postulaciones_Marketing
0,1112257047,NM5M,True,Atención al Cliente,False,True,False,False,True,False,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1111920714,NM5M,True,Telemarketing,False,True,True,False,False,False,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1112346945,NM5M,True,Telemarketing,True,False,True,False,False,False,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1112345547,NM5M,True,Telemarketing,True,False,True,False,False,False,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1112237522,5awk,True,Contabilidad,True,False,True,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [80]:
postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6603752 entries, 0 to 6603751
Data columns (total 56 columns):
idaviso                                          int64
idpostulante                                     object
GBA                                              object
nombre_area                                      object
Full-time                                        object
Part-time                                        object
senior                                           object
junior                                           object
otro                                             object
Jefe                                             object
Gerencia                                         object
Ventas                                           object
Comercial                                        object
Administración                                   object
Producción                                       object
Programación                                    

Por alguna razon que desconozco, columnas que antes eran booleanas se transformaron en object
y aumento mucho el tamaño de los datos. Vamos a transformarlas nuevamente en booleanas

In [81]:
columnas_booleanas = ['GBA','Full-time',
       'Part-time', 'senior', 'junior', 'otro', 'Jefe', 'Gerencia', 'Ventas',
       'Comercial', 'Administración', 'Producción', 'Programación',
       'Contabilidad', 'Tecnologia / Sistemas', 'Atención al Cliente',
       'Mantenimiento', 'Recursos Humanos', 'Logística', 'Gastronomia',
       'Call Center', 'Soporte Técnico', 'Almacén / Depósito / Expedición',
        'Oficios y Profesiones','Marketing',
       'postulacion','sexo_masculino', 'titulo_universitario',
       'titulo_secundario', 'titulo_superior']


In [82]:
for columna in columnas_booleanas:
    postulaciones[columna] = postulaciones[columna].astype(bool)
    
postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6603752 entries, 0 to 6603751
Data columns (total 56 columns):
idaviso                                          int64
idpostulante                                     object
GBA                                              bool
nombre_area                                      object
Full-time                                        bool
Part-time                                        bool
senior                                           bool
junior                                           bool
otro                                             bool
Jefe                                             bool
Gerencia                                         bool
Ventas                                           bool
Comercial                                        bool
Administración                                   bool
Producción                                       bool
Programación                                     bool
Contabilidad      

Agregamos a las postulaciones la cantidad de visitas que cada usuario realizo al aviso
al que se postuló

In [83]:
postulaciones = postulaciones.merge(concatVistas,
                                    on = ['idaviso','idpostulante'],
                                    how = 'left')
postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6603752 entries, 0 to 6603751
Data columns (total 57 columns):
idaviso                                          int64
idpostulante                                     object
GBA                                              bool
nombre_area                                      object
Full-time                                        bool
Part-time                                        bool
senior                                           bool
junior                                           bool
otro                                             bool
Jefe                                             bool
Gerencia                                         bool
Ventas                                           bool
Comercial                                        bool
Administración                                   bool
Producción                                       bool
Programación                                     bool
Contabilidad      

In [84]:
postulaciones['visitas'].fillna(0,inplace = True)
postulaciones.head()

,idaviso,idpostulante,GBA,nombre_area,Full-time,Part-time,senior,junior,otro,Jefe,...,postulaciones_Mantenimiento,postulaciones_Recursos Humanos,postulaciones_Logística,postulaciones_Gastronomia,postulaciones_Call Center,postulaciones_Soporte Técnico,postulaciones_Almacén / Depósito / Expedición,postulaciones_Oficios y Profesiones,postulaciones_Marketing,visitas
0,1112257047,NM5M,True,Atención al Cliente,False,True,False,False,True,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1111920714,NM5M,True,Telemarketing,False,True,True,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1112346945,NM5M,True,Telemarketing,True,False,True,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1112345547,NM5M,True,Telemarketing,True,False,True,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1112237522,5awk,True,Contabilidad,True,False,True,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [85]:
postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6603752 entries, 0 to 6603751
Data columns (total 57 columns):
idaviso                                          int64
idpostulante                                     object
GBA                                              bool
nombre_area                                      object
Full-time                                        bool
Part-time                                        bool
senior                                           bool
junior                                           bool
otro                                             bool
Jefe                                             bool
Gerencia                                         bool
Ventas                                           bool
Comercial                                        bool
Administración                                   bool
Producción                                       bool
Programación                                     bool
Contabilidad      

In [86]:
postulaciones.isnull().sum()

idaviso                                               0
idpostulante                                          0
GBA                                                   0
nombre_area                                      415823
Full-time                                             0
Part-time                                             0
senior                                                0
junior                                                0
otro                                                  0
Jefe                                                  0
Gerencia                                              0
Ventas                                                0
Comercial                                             0
Administración                                        0
Producción                                            0
Programación                                          0
Contabilidad                                          0
Tecnologia / Sistemas                           

In [87]:
postulaciones.dropna(inplace = True)
postulaciones.isnull().sum()

idaviso                                          0
idpostulante                                     0
GBA                                              0
nombre_area                                      0
Full-time                                        0
Part-time                                        0
senior                                           0
junior                                           0
otro                                             0
Jefe                                             0
Gerencia                                         0
Ventas                                           0
Comercial                                        0
Administración                                   0
Producción                                       0
Programación                                     0
Contabilidad                                     0
Tecnologia / Sistemas                            0
Atención al Cliente                              0
Mantenimiento                  

Aqui ya tenemos un set de datos que nos serviría para entrenar a un algoritmo 
de Machine Learning. Este algoritmo debe predecir si un postulante se postuló o no a un 
determinado aviso basandose en los features que definimos. Se podria decir que se trata
de un problema de clasificación binaria, donde las categorias posibles de clasificación son
'postulacion' o 'no_postulacion'. 
Bajo esta caracterización del problema, ahora nuestro set de entrenamiento resulta incompleto,
ya que solo tenemos datos de una categoría ('postulacion' o columna 'postulacion' = True).
Por lo tanto debemos generar datos que correspondan a pares de usuarios y avisos cuya 
categoría corresponda a 'no_postulacion'.

In [88]:
postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6187929 entries, 0 to 6603751
Data columns (total 57 columns):
idaviso                                          int64
idpostulante                                     object
GBA                                              bool
nombre_area                                      object
Full-time                                        bool
Part-time                                        bool
senior                                           bool
junior                                           bool
otro                                             bool
Jefe                                             bool
Gerencia                                         bool
Ventas                                           bool
Comercial                                        bool
Administración                                   bool
Producción                                       bool
Programación                                     bool
Contabilidad      

## Generando No_Postulaciones

In [89]:
no_postulaciones = pd.read_csv('data/no_postulaciones.csv')

In [90]:
no_postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7479162 entries, 0 to 7479161
Data columns (total 3 columns):
Unnamed: 0      int64
idpostulante    object
idaviso         int64
dtypes: int64(2), object(1)
memory usage: 171.2+ MB


In [91]:
no_postulaciones.drop(columns = 'Unnamed: 0', inplace = True)
no_postulaciones.head()

,idpostulante,idaviso
0,YjQ9511,1111438862
1,vVeGPY5,1112346693
2,EWv5r4,1112230559
3,5mrwNPq,1112377508
4,2zPLrpw,1112513501


In [92]:
no_postulaciones = no_postulaciones.merge(avisos, on = 'idaviso', how = 'left')
no_postulaciones.head()

,idpostulante,idaviso,GBA,nombre_area,Full-time,Part-time,senior,junior,otro,Jefe,...,Logística,Gastronomia,Call Center,Soporte Técnico,Almacén / Depósito / Expedición,Oficios y Profesiones,Marketing,postulaciones_aviso,postulaciones_masculinas,postulaciones_universitarias
0,YjQ9511,1111438862,True,Atención al Cliente,True,False,False,True,False,False,...,False,False,False,False,False,False,False,0.0,0.000000,0.000000
1,vVeGPY5,1112346693,True,Construcción,True,False,False,False,True,False,...,False,False,False,False,False,False,False,19.0,0.947368,0.315789
2,EWv5r4,1112230559,True,Logística,True,False,False,False,True,False,...,True,False,False,False,False,False,False,19.0,0.947368,0.263158
3,5mrwNPq,1112377508,True,Trabajo social,True,False,False,False,False,True,...,False,False,False,False,False,False,False,55.0,0.418182,0.636364
4,2zPLrpw,1112513501,True,Contabilidad,True,False,False,True,False,False,...,False,False,False,False,False,False,False,0.0,0.000000,0.000000


In [93]:
no_postulaciones = no_postulaciones.merge(usuarios, on = 'idpostulante', how = 'left')
no_postulaciones.head()

,idpostulante,idaviso,GBA,nombre_area,Full-time,Part-time,senior,junior,otro,Jefe,...,postulaciones_Atención al Cliente,postulaciones_Mantenimiento,postulaciones_Recursos Humanos,postulaciones_Logística,postulaciones_Gastronomia,postulaciones_Call Center,postulaciones_Soporte Técnico,postulaciones_Almacén / Depósito / Expedición,postulaciones_Oficios y Profesiones,postulaciones_Marketing
0,YjQ9511,1111438862,True,Atención al Cliente,True,False,False,True,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,vVeGPY5,1112346693,True,Construcción,True,False,False,False,True,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,EWv5r4,1112230559,True,Logística,True,False,False,False,True,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5mrwNPq,1112377508,True,Trabajo social,True,False,False,False,False,True,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2zPLrpw,1112513501,True,Contabilidad,True,False,False,True,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [94]:
no_postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7479162 entries, 0 to 7479161
Data columns (total 55 columns):
idpostulante                                     object
idaviso                                          int64
GBA                                              bool
nombre_area                                      object
Full-time                                        bool
Part-time                                        bool
senior                                           bool
junior                                           bool
otro                                             bool
Jefe                                             bool
Gerencia                                         bool
Ventas                                           bool
Comercial                                        bool
Administración                                   bool
Producción                                       bool
Programación                                     bool
Contabilidad      

In [95]:
no_postulaciones = no_postulaciones.merge(concatVistas,
                                    on = ['idaviso','idpostulante'],
                                    how = 'left')
no_postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7479162 entries, 0 to 7479161
Data columns (total 56 columns):
idpostulante                                     object
idaviso                                          int64
GBA                                              bool
nombre_area                                      object
Full-time                                        bool
Part-time                                        bool
senior                                           bool
junior                                           bool
otro                                             bool
Jefe                                             bool
Gerencia                                         bool
Ventas                                           bool
Comercial                                        bool
Administración                                   bool
Producción                                       bool
Programación                                     bool
Contabilidad      

In [96]:
no_postulaciones['postulacion'] = False

In [97]:
no_postulaciones['visitas'].fillna(0, inplace = True)
no_postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7479162 entries, 0 to 7479161
Data columns (total 57 columns):
idpostulante                                     object
idaviso                                          int64
GBA                                              bool
nombre_area                                      object
Full-time                                        bool
Part-time                                        bool
senior                                           bool
junior                                           bool
otro                                             bool
Jefe                                             bool
Gerencia                                         bool
Ventas                                           bool
Comercial                                        bool
Administración                                   bool
Producción                                       bool
Programación                                     bool
Contabilidad      

In [98]:
postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6187929 entries, 0 to 6603751
Data columns (total 57 columns):
idaviso                                          int64
idpostulante                                     object
GBA                                              bool
nombre_area                                      object
Full-time                                        bool
Part-time                                        bool
senior                                           bool
junior                                           bool
otro                                             bool
Jefe                                             bool
Gerencia                                         bool
Ventas                                           bool
Comercial                                        bool
Administración                                   bool
Producción                                       bool
Programación                                     bool
Contabilidad      

In [99]:
no_postulaciones = no_postulaciones[postulaciones.columns]

Las no postulaciones ya tienen el mismo formato que las postulaciones, con la columna que 
corresponde a las postulaciones en False.
Vamos a concatenar postulaciones y no_postulaciones en el DF de postulaciones, para tener
un solo set de datos compacto

In [100]:
postulaciones = pd.concat([postulaciones,no_postulaciones])

In [101]:
postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13667091 entries, 0 to 7479161
Data columns (total 57 columns):
idaviso                                          int64
idpostulante                                     object
GBA                                              bool
nombre_area                                      object
Full-time                                        bool
Part-time                                        bool
senior                                           bool
junior                                           bool
otro                                             bool
Jefe                                             bool
Gerencia                                         bool
Ventas                                           bool
Comercial                                        bool
Administración                                   bool
Producción                                       bool
Programación                                     bool
Contabilidad     

Ahora si, ya tenemos nuestro set de datos listo para entrenar un algoritmo de Machine Learning

In [102]:
postulaciones.isnull().sum()

idaviso                                          0
idpostulante                                     0
GBA                                              0
nombre_area                                      0
Full-time                                        0
Part-time                                        0
senior                                           0
junior                                           0
otro                                             0
Jefe                                             0
Gerencia                                         0
Ventas                                           0
Comercial                                        0
Administración                                   0
Producción                                       0
Programación                                     0
Contabilidad                                     0
Tecnologia / Sistemas                            0
Atención al Cliente                              0
Mantenimiento                  

# Importando el set de datos corresponiente al Test final

Importamos el csv test_final. Que tiene formato (id,idaviso,idpostulante).
Debemos darle el mismo formato que tiene el df postulaciones, agregando los features
correspondientes. Es claro que este df tendrá una columna menos, la de postulaciones, ya que 
esa es la columna que el algoritmo de ML deberá predecir.

In [103]:
test_final = pd.read_csv('data/test_final_100k.csv')

In [104]:
test_final = test_final.merge(usuarios, on='idpostulante', how = 'left')
test_final.head()

,id,idaviso,idpostulante,edad,sexo_masculino,titulo_universitario,titulo_secundario,titulo_superior,cantidad_postulaciones,postulaciones_gba,...,postulaciones_Atención al Cliente,postulaciones_Mantenimiento,postulaciones_Recursos Humanos,postulaciones_Logística,postulaciones_Gastronomia,postulaciones_Call Center,postulaciones_Soporte Técnico,postulaciones_Almacén / Depósito / Expedición,postulaciones_Oficios y Profesiones,postulaciones_Marketing
0,0,739260,6M9ZQR,42.0,False,True,False,True,2.0,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,739260,6v1xdL,30.0,True,False,False,False,68.0,56,...,14.0,0.0,0.0,1.0,3.0,4.0,0.0,0.0,0.0,0.0
2,2,739260,ezRKm9,36.0,False,True,False,False,1.0,True,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,758580,1Q35ej,68.0,True,True,False,True,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,758580,EAN4J6,32.0,False,False,False,False,1.0,True,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [105]:
test_final = test_final.merge(avisos,
                            on = 'idaviso',
                            how = 'left')
test_final.head()

,id,idaviso,idpostulante,edad,sexo_masculino,titulo_universitario,titulo_secundario,titulo_superior,cantidad_postulaciones,postulaciones_gba,...,Logística,Gastronomia,Call Center,Soporte Técnico,Almacén / Depósito / Expedición,Oficios y Profesiones,Marketing,postulaciones_aviso,postulaciones_masculinas,postulaciones_universitarias
0,0,739260,6M9ZQR,42.0,False,True,False,True,2.0,2,...,False,False,False,False,False,False,False,0.0,0.0,0.0
1,1,739260,6v1xdL,30.0,True,False,False,False,68.0,56,...,False,False,False,False,False,False,False,0.0,0.0,0.0
2,2,739260,ezRKm9,36.0,False,True,False,False,1.0,True,...,False,False,False,False,False,False,False,0.0,0.0,0.0
3,3,758580,1Q35ej,68.0,True,True,False,True,0.0,0,...,False,False,False,False,False,False,False,0.0,0.0,0.0
4,4,758580,EAN4J6,32.0,False,False,False,False,1.0,True,...,False,False,False,False,False,False,False,0.0,0.0,0.0


In [106]:
test_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 56 columns):
id                                               100000 non-null int64
idaviso                                          100000 non-null int64
idpostulante                                     100000 non-null object
edad                                             100000 non-null float64
sexo_masculino                                   100000 non-null bool
titulo_universitario                             100000 non-null bool
titulo_secundario                                100000 non-null bool
titulo_superior                                  100000 non-null bool
cantidad_postulaciones                           100000 non-null float64
postulaciones_gba                                100000 non-null object
postulaciones_Ventas                             100000 non-null float64
postulaciones_Comercial                          100000 non-null float64
postulaciones_Administración    

In [107]:
test_final = test_final.merge(concatVistas, on = ['idaviso','idpostulante'], how = 'left')
test_final.head()

,id,idaviso,idpostulante,edad,sexo_masculino,titulo_universitario,titulo_secundario,titulo_superior,cantidad_postulaciones,postulaciones_gba,...,Gastronomia,Call Center,Soporte Técnico,Almacén / Depósito / Expedición,Oficios y Profesiones,Marketing,postulaciones_aviso,postulaciones_masculinas,postulaciones_universitarias,visitas
0,0,739260,6M9ZQR,42.0,False,True,False,True,2.0,2,...,False,False,False,False,False,False,0.0,0.0,0.0,NaN
1,1,739260,6v1xdL,30.0,True,False,False,False,68.0,56,...,False,False,False,False,False,False,0.0,0.0,0.0,NaN
2,2,739260,ezRKm9,36.0,False,True,False,False,1.0,True,...,False,False,False,False,False,False,0.0,0.0,0.0,NaN
3,3,758580,1Q35ej,68.0,True,True,False,True,0.0,0,...,False,False,False,False,False,False,0.0,0.0,0.0,NaN
4,4,758580,EAN4J6,32.0,False,False,False,False,1.0,True,...,False,False,False,False,False,False,0.0,0.0,0.0,NaN


In [108]:
test_final['visitas'].fillna(0, inplace = True)
test_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 57 columns):
id                                               100000 non-null int64
idaviso                                          100000 non-null int64
idpostulante                                     100000 non-null object
edad                                             100000 non-null float64
sexo_masculino                                   100000 non-null bool
titulo_universitario                             100000 non-null bool
titulo_secundario                                100000 non-null bool
titulo_superior                                  100000 non-null bool
cantidad_postulaciones                           100000 non-null float64
postulaciones_gba                                100000 non-null object
postulaciones_Ventas                             100000 non-null float64
postulaciones_Comercial                          100000 non-null float64
postulaciones_Administración    

Ahora tenemos todo listo para entrenar nuestro algoritmo de ML

# Empieza perceptron


La columna 'nombre_area' no nos va a servir para este algoritmo, la vamos a dejar de lado.
Lo mismo con los ids, que no son features en sí

In [109]:
datos = postulaciones.drop(columns = ['idpostulante','idaviso','nombre_area','postulacion'])
categorias = postulaciones['postulacion']


In [110]:
columnas_numeradas = pd.DataFrame(data = datos.columns)
columnas_numeradas

,0
0,GBA
1,Full-time
2,Part-time
3,senior
4,junior
5,otro
6,Jefe
7,Gerencia
8,Ventas
9,Comercial


In [264]:
test_size = 0.4
random_state = 0

In [265]:
datos_train, datos_test, cat_train, cat_test = train_test_split(datos,
                                                                categorias,
                                                                test_size = test_size,
                                                                random_state = random_state)

In [266]:
sc = StandardScaler()
sc.fit(datos_train)

datos_train_std = sc.transform(datos_train)
datos_test_std = sc.transform(datos_test)


In [267]:
datos_train_final = datos_train_std
datos_test_final = datos_test_std

In [268]:
n_iter=40
eta0 = 0.1

In [167]:
#ppn con todos los features
ppn = Perceptron(max_iter = n_iter,
                eta0 = eta0,
                random_state = random_state)

ppn.fit(datos_train_final,cat_train)



Perceptron(alpha=0.0001, class_weight=None, eta0=0.1, fit_intercept=True,
      max_iter=40, n_iter=None, n_jobs=1, penalty=None, random_state=0,
      shuffle=True, tol=None, verbose=0, warm_start=False)

In [168]:
cat_pred = ppn.predict(datos_test_final)

print('accuracy: {0:.2f}%'.format(accuracy_score(cat_test,cat_pred)*100))

accuracy: 80.62%


### Perceptron con menos features

Aquí entrenamos otro Perceptron utilizando menos features de nuestro set de datos. Esto nos sirve para identificar que features son realmente relevantes y cuales quizas no aportan mucho al entrenamiento y la predicción

In [284]:
columnas = [25,33,52]

In [285]:
datos_train_final = datos_train_std[:,columnas]
datos_test_final = datos_test_std[:,columnas]

In [286]:
n_iter=40
eta0 = 0.1

ppn2 = Perceptron(max_iter = n_iter,
                eta0 = eta0,
                random_state = random_state)

ppn2.fit(datos_train_final,cat_train)



Perceptron(alpha=0.0001, class_weight=None, eta0=0.1, fit_intercept=True,
      max_iter=80, n_iter=None, n_jobs=1, penalty=None, random_state=0,
      shuffle=True, tol=None, verbose=0, warm_start=False)

In [287]:
cat_pred = ppn2.predict(datos_test_final)

print('accuracy: {0:.2f}%'.format(accuracy_score(cat_test,cat_pred)*100))

accuracy: 83.79%


In [288]:
final_test_data = test_final[datos.columns]

In [289]:
final_test_data.head()

,GBA,Full-time,Part-time,senior,junior,otro,Jefe,Gerencia,Ventas,Comercial,...,postulaciones_Mantenimiento,postulaciones_Recursos Humanos,postulaciones_Logística,postulaciones_Gastronomia,postulaciones_Call Center,postulaciones_Soporte Técnico,postulaciones_Almacén / Depósito / Expedición,postulaciones_Oficios y Profesiones,postulaciones_Marketing,visitas
0,False,True,False,False,False,False,True,False,False,True,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,False,True,False,False,False,False,True,False,False,True,...,0.0,0.0,1.0,3.0,4.0,0.0,0.0,0.0,0.0,0.0
2,False,True,False,False,False,False,True,False,False,True,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,False,True,False,False,False,True,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,False,True,False,False,False,True,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [290]:
final_test_data = sc.transform(final_test_data)
final_test_data = final_test_data[:,columnas]

In [291]:
prediccion = ppn2.predict(final_test_data)
prediccion = prediccion.astype(int)

In [292]:
test_final['sepostulo'] = prediccion
entregable = test_final[['id','sepostulo']]

In [293]:
entregable.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 2 columns):
id           100000 non-null int64
sepostulo    100000 non-null int64
dtypes: int64(2)
memory usage: 2.3 MB


In [294]:
entregable.index = entregable['id']
entregable.drop(columns = 'id', inplace = True)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [297]:
entregable2 = pd.read_csv('submits/submit_28.csv')

In [298]:
print('accuracy: {0:.2f}%'.format(accuracy_score(entregable['sepostulo'],
                                                 entregable2['sepostulo'])*100))

accuracy: 89.20%
